In [1]:
import pandas as pd

In [41]:
import warnings
warnings.filterwarnings("ignore")

Here we take a subsample of the store data from Kaggle 

In [2]:
train_df = pd.read_csv("data/store/subset.csv")

This data set has 10 * 3 = 30 time series.

### Building an ARIMA model 

In [36]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

Because ARIMA models are univariate models, we need to build each model for each time series separately (30 models). Here we just build a model on one of the time series 

Let's first visualize this time series

In [25]:
import seaborn as sns

split the data into train and valid dataset

Model training - let's choose some p, d, q to start with

plot the model training result -- we will plot both the confidence interval and mean of the predictions

Evaluate the models using root mean squared error

In [118]:
from sklearn.metrics import mean_squared_error


Since this model is not in Scikit-learn, we cannot use Scikit-learn's grid search function. Let's create our own grid search function to find the best p, d, q

We need to repeat this process for ALL 30 time series. Quiz: write a python functionn to perform the above process (train ARIMA model and find the best hyperparameter) for any time series, and run the function on the other 29 time series 